# SubmittableTextBox

`SubmittableTextBox` is a special `Text` widget supporting key strokes events and text submissions.

Its methods `get_key_stroke_future()` and `get_submitted_value_future()` return an awaitable set to respectively the observed and the validated text.

Both methods are meant to be awaited continuously. In the snippet below, key strokes and text submissions are displayed in an `Output` widget.

In [1]:
from here_search.widgets.request import SubmittableTextBox
from ipywidgets import Output
from asyncio import ensure_future

box = SubmittableTextBox()
layout_30 = {'height': '30px', 'border': '1px solid black'}
out_keys = Output(layout=layout_30)
out_text = Output(layout=layout_30)

async def print_key_strokes(box, out):
    while True:
        with out:
            keys = await box.get_key_stroke_future()
            out.outputs = ({'name': 'stdout', 'output_type': 'stream', 'text': keys}, )

async def print_submitted_text(box, out):
    while True:
        with out:
            text = await box.get_submitted_value_future()
            out.outputs = ({'name': 'stdout', 'output_type': 'stream', 'text': text}, )            
            
t1 = ensure_future(print_key_strokes(box, out_keys))
t2 = ensure_future(print_submitted_text(box, out_text))
display(box, out_keys, out_text)

HTML(value='<style>.term-button button { font-size: 10px; }</style>')

SubmittableTextBox(children=(SubmittableText(value=''), Button(icon='search', layout=Layout(width='32px'), sty…

Output(layout=Layout(border='1px solid black', height='30px'))

Output(layout=Layout(border='1px solid black', height='30px'))

## TermsButtons

`TermsButtons` objects are taking a `SubmittableTextBox` instance to feed it with predefined values when the buttons are selected. Per default, the complete form value is overwritten by the button text. 

The constructor `index` parameter allows to specify wich token is overwritten:

In [2]:
from here_search.widgets.request import TermsButtons

box2 = SubmittableTextBox()
out_keys2 = Output(layout=layout_30)

terms = TermsButtons(box2, 
                     values=["text 1", "text 2"], 
                     index=-1)

t3 = ensure_future(print_key_strokes(box2, out_keys2))
display(box2, terms, out_keys2)

SubmittableTextBox(children=(SubmittableText(value=''), Button(icon='search', layout=Layout(width='32px'), sty…

TermsButtons(children=(Button(description='text 1', layout=Layout(border='solid 1px', display='flex', justify_…

Output(layout=Layout(border='1px solid black', height='30px'))

# OneBoxSimple

Similarly to the previous example, the `OneBoxSimple` objects continuously wait for both key strokes and text submissions through `wait_for_new_key_stroke()` and `wait_for_submitted_value()` methods. 

`OneBoxSimple` then sends keystrokes and text submissions to Autosuggest and Discover respectively. The related responses are handled through `handle_suggestion_list()` and `handle_result_list()` methods.

In [3]:
from here_search.base import OneBoxSimple

In [4]:
from IPython.display import Code
from inspect import getsource
#Code(getsource(OneBoxSimple), language="python")

In the snippet below, `App` uses a `SubmittableTextBox` and an `Output` widgets to implement missing methods of `OneBoxSimple`:

In [8]:
from here_search.base import OneBoxSimple
from here_search.widgets.request import SubmittableTextBox
from asyncio import Future
from ipywidgets import Output
from json import dumps

box = SubmittableTextBox()
out = Output(layout={'height': '160px', 'border': '1px solid black', 'overflow': 'auto', 'white-space': 'nowrap'})

class App(OneBoxSimple):
    default_language = "de"
    
    def wait_for_new_key_stroke(self):
        return box.get_key_stroke_future()
    
    def wait_for_submitted_value(self):
        return box.get_submitted_value_future()
    
    def handle_suggestion_list(self, resp):
        with out:
            out.outputs = ({'name': 'stdout', 'output_type': 'stream', 'text': dumps(resp.data, indent=2)}, )
            
    def handle_result_list(self, resp):
        self.handle_suggestion_list(resp)
        box.text_w.value = ''

App().run()
display(box, out)

SubmittableTextBox(children=(SubmittableText(value=''), Button(icon='search', layout=Layout(width='32px'), sty…

Output(layout=Layout(border='1px solid black', height='160px', overflow='auto'))

With `TermsButtons`, one has:

In [9]:
box2 = SubmittableTextBox()
terms2 = TermsButtons(box2, index=-1)
out2 = Output(layout={'height': '160px', 'border': '1px solid black', 'overflow': 'auto', 'white-space': 'nowrap'})

class App2(App):
    
    def wait_for_new_key_stroke(self):
        return box2.get_key_stroke_future()
    
    def wait_for_submitted_value(self):
        return box2.get_submitted_value_future()
    
    def handle_suggestion_list(self, resp):
        with out2:
            out2.outputs = ({'name': 'stdout', 'output_type': 'stream', 'text': dumps(resp.data, indent=2)}, )
        text = {term['term']: None for term in resp.data.get('queryTerms', [])}
        terms2.set(list(text.keys()))
            
    def handle_result_list(self, resp):
        self.handle_suggestion_list(resp)
        box2.text_w.value = ''
        terms2.set([])
        
display(box2, terms2, out2)
App2().run()

SubmittableTextBox(children=(SubmittableText(value=''), Button(icon='search', layout=Layout(width='32px'), sty…

TermsButtons(children=(Button(description=' ', layout=Layout(border='solid 1px', display='flex', justify_conte…

Output(layout=Layout(border='1px solid black', height='160px', overflow='auto'))

# OntologyButtons

TBC

## PositionMap

`OneBoxSimple` always uses the same search center, hosted as a class attribute. It is convenient to use the center of a map to gather a position.

The `PositionMap` class uses [`here-map-widget`](https://pypi.org/project/here-map-widget-for-jupyter/) dragging and zooming capabilities to capture the center of a HERE map and sends it to an observer. 

For this it expects a `position_handler` constructor parameter respecting the `traitlets` event handler [signature](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html#Registering-callbacks-to-trait-changes-in-the-kernel). In the snippet below, the map position is only displayed in an `Output` widget, each time the user interacts with the map:

In [10]:
from here_search.widgets.response import PositionMap
from here_search.api import API

out = Output(layout = {'width': '100%', 'height': '30px', 'overflow': 'auto', 'white-space': 'nowrap'})

def observe(change):
    if change.type == "change" and change.new:
        with out:
            out.outputs = ({'name': 'stdout', 'output_type': 'stream', 'text': dumps({"name": change.name, "new": change.new})}, )
    
pmap = PositionMap(api_key=API().api_key, 
                   center=(52.51604, 13.37691), 
                   position_handler=observe)
display(pmap, out)

PositionMap(api_key='(...)', basemap=TileLayer(provider=MapTile(format='…

Output(layout=Layout(height='30px', overflow='auto', width='100%'))

# OneBoxMap


`OneBoxMap` is a demo application showing how [HERE Geocoding and Search](https://developer.here.com/documentation/geocoding-search-api/dev_guide/topics/endpoint-autosuggest-brief.html) `/autosuggest`, `/discover` and `/lookup` endpoints are meant to be used:
- The app  
    - proposes a single text form to formulate queries
    - displays three buttons with predicted last token text completions
    - displays all API calls in a separate window
    - displays search results on a map and a list
<br/>
- `/autosuggest` endpoint is called for each key-stroke.
    - Follow-up query suggestions are displayed in the list only
    - Location suggestions are displayed in the list and on the map
    - Query term suggestions are displayed in three buttons
    - Selected locations lead to a call to `/lookup` using the location record `id`.
    - Selected query suggestions lead to a http GET using the query `href` value
<br/>
- `/discover` endpoint is called when a query submission is validated.
    - Validation happens when the [return] key or the <img src="https://upload.wikimedia.org/wikipedia/commons/2/2b/Font_Awesome_5_solid_search.svg" style="width:12px"/> button are hit
    - Location results are displayed in the list and on the map
    - Selected results lead to a call to `/lookup` using the location record `id`.
<br/>
- `/autosuggest` and `/discover` requests are sent using the map center as search center
- Search requests are cached during the lifetime of the application


<!-- https://commons.wikimedia.org/wiki/File:Font_Awesome_5_solid_search.svg -->

In [11]:
from here_search.widgets.app import OneBoxMap
OneBoxMap().run()

HTML(value='<style>.result-button div, .result-button button { font-size: 10px; }</style>')

PositionMap(api_key='(...)', basemap=TileLayer(provider=MapTile(format='…

Output(layout=Layout(border='1px solid black', height='160px', overflow='auto', width='100%'))